<a href="https://colab.research.google.com/github/maddogmikeb/Jira/blob/master/TimeInStatus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# install dependencies

!pip install -q atlassian-python-api

In [21]:
# Configure display

from google.colab import data_table
data_table.enable_dataframe_formatter()

In [22]:
# Log in

from IPython.core.display import display, HTML
from atlassian import Jira
from google.colab import userdata

jira = Jira(
  url=userdata.get('atlassian_host'),
  username=userdata.get('atlassian_username'),
  password=userdata.get('atlassian_apikey'),
  cloud=True
)

me = jira.myself()

display(HTML('<table><tr><td>' + me["displayName"] + '</td><td><img src="' + me["avatarUrls"]["32x32"] + '"/><td></tr></table>'))


Mike Burns


In [23]:
# Get all issues from jql
# Code to help debug -> https://github.com/atlassian-api/atlassian-python-api/blob/master/atlassian/jira.py

from IPython.display import clear_output, display
from atlassian import Jira

JQL = 'project = FDSEWMSR AND issuetype in (Bug, Story, Epic, Task) AND "Team[Team]" = d3706851-4fae-4b34-9a25-d4e10c5a45e4 and statuscategory = "done" ORDER BY Rank ASC'

limit = None

params = {}
if limit is not None:
  params["maxResults"] = int(limit)
params["fields"] = "key,created,resolutiondate,status,project"
params["jql"] = JQL
#params["expand"] = expand
url = jira.resource_url("search")
start = 0
results = []

while True:
  clear_output(wait=True)

  params["startAt"] = int(start)
  response = jira.get(url, params=params)
  if not response:
    break

  issues = response["issues"]
  results.extend(issues)
  total = int(response["total"])
  display("DBG: response: total={total} start={startAt} max={maxResults}".format(**response))
  # If we don't have a limit, and there's more to fetch, keep looping
  if limit is not None or total <= len(response["issues"]) + start:
    break
  start += len(issues)

clear_output()

In [24]:
# Get all the change logs and iterate through them to find all the status changes

from IPython.display import clear_output, display
from atlassian import Jira
import datetime
import pandas as pd
import numpy as np
import copy

issues = copy.deepcopy(results)

for issue in issues:
  clear_output(wait=True)
  display("DBG: checking key={key}".format(**issue))

  changelog = jira.get_issue_changelog(issue["key"])
  changes = []
  lastChange = issue["fields"]["created"]
  for log in changelog["values"]:
    for logitem in log["items"]:
      if logitem["field"].upper() == "STATUS":
        logitem["start"] = lastChange
        logitem["end"] = log["created"]
        lastChange = log["created"]
        changes += [
            {
              'statusid': logitem["from"],
              'status': logitem["fromString"],
              #'start': logitem["start"],
              #'end': logitem["end"],
              'total': (datetime.datetime.fromisoformat(logitem["end"]) - datetime.datetime.fromisoformat(logitem["start"])).total_seconds()
            }]
        #display(logitem)
  if len(changes) > 0:
    changes += [
      {
        'statusid': issue["fields"]["status"]["id"],
        'status': issue["fields"]["status"]["name"],
        #'start': lastChange,
        #'end': None,
        'total': float('inf')
      }]

    df = pd.DataFrame(changes)
    df.groupby('statusid', as_index=False)['total'].sum()
    df = df.reset_index()
    df = df.replace({None: np.nan})
    for index, row in df.iterrows():
      issue[row["statusid"] + "|" + row["status"]] = row["total"]

  # clean up fields
  issue["created"] = issue["fields"]["created"]
  issue["resolutiondate"] = issue["fields"]["resolutiondate"]
  issue["project"] = issue["fields"]["project"]["name"]
  issue["url"] = jira.url + "browse/" + issue["key"]
  del issue["fields"]
  del issue["expand"]
  del issue["self"]

clear_output()
#display( pd.DataFrame(changes) )

In [25]:
# Print

from IPython.core.display import display, HTML

import json
import pandas as pd
import numpy as np

#print(json.dumps(data, indent=2))

df = pd.DataFrame(issues)
df = df.reindex(sorted(df.columns, reverse=True), axis=1)
df = df.replace({None: np.nan})
# display(df)

df.to_excel("output.xlsx", index=False)